In [9]:
class ShoppingCart(object):

    def __init__(self):
        self.total = 0
        self.items = {}

    def add_items(self, item_name, quantity, price):
        self.total = self.total + quantity * price
        self.items.update(
            {item_name: quantity}
        )
    
    def remove_items(self, item_name, quantity, price):
        self.total = self.total - (quantity * price)
        if quantity >= self.items[item_name]:
            del self.items[item_name]
        else:
            self.items[item_name] -= quantity
        
    def checkout(self, cash_paid):
        balance = 0
        if cash_paid < self.total:
            return "You paid {} but the total amount is {}".format(cash_paid, self.total)
        #total = 45, cash_paid = 100
        balance = cash_paid - self.total
        return "Exchange amount is {}".format(balance)


In [10]:
cart = ShoppingCart()
cart.add_items("banana", 3, 0.5)
cart.add_items("salad kit", 1, 5)
cart.add_items("cod fillet", 2, 8)

In [12]:
cart.items

{'banana': 3, 'salad kit': 1, 'cod fillet': 2}

In [13]:
cart.remove_items("banana", 1, 0.5)

In [14]:
cart.items

{'banana': 2, 'salad kit': 1, 'cod fillet': 2}

In [15]:
cart_result = cart.checkout(100)
cart_result

'Exchange amount is 78.0'

### Encapsulation

In [16]:
class BankAccount:
    def __init__(self, account_number, balance):
        self.account_number = account_number   # Public Attribute
        self.__balance = balance              # Private Attribute

    # Getter method
    def get_balance(self):
        return self.__balance

    # Setter method
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
        else:
            print("Invalid deposit amount")

    # Private Method
    def __secret_method(self):
        print("This is a secret method")


In [17]:
account = BankAccount("HH1207", 100)

In [24]:
account.__balance 
#this will error out because __ nmakes them hidden from outside the class 

AttributeError: 'BankAccount' object has no attribute '__balance'

In [22]:
account.get_balance()

100

### Abstraction

Abstraction is another key concept in Object-Oriented Programming (OOP). It is the process of hiding unnecessary implementation details and showing only the essential features of an object.

In [29]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    @abstractmethod
    def park(self):
        pass

class CompactCar(Vehicle):
    def park(self):
        print("compact car is parking in a compact spot")

class Motorcycle(Vehicle):
    def park(self):
        print("motorcycle is parking in a motorcycle spot")


In [30]:
vehicles = [CompactCar(), Motorcycle()]
for v in vehicles:
    v.park()

compact car is parking in a compact spot
motorcycle is parking in a motorcycle spot


### Polymorphism

Different objects can be treated the same way throiugh a common interface

In [182]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    def __init__(self, license_plate):
        self.license_plate = license_plate  # Encapsulation (private-like attribute)

    @abstractmethod
    def park(self):
        pass

    def display_plate(self):
        print(f"Vehicle with plate {self.license_plate} is parking")

class CompactCar(Vehicle):
    def park(self):
        self.display_plate()
        print("Compact car is parking in a compact spot")

class Motorcycle(Vehicle):
    def park(self):
        self.display_plate()
        print("Motorcycle is parking in a motorcycle spot")

class Truck(Vehicle):
    def park(self):
        self.display_plate()
        print("Truck is parking in a large spot")


In [233]:
class ParkingSpot:

    def __init__(self, spot_id, distance, vehicle_type):

        self.spot_id = spot_id
        self.distance = distance
        #check that vehicle type is an instance of the class Vehicle

        if not issubclass(vehicle_type, Vehicle):
            raise ValueError("vehicle_type must be a subclass of a vehicle")
        
        self.vehicle_type = vehicle_type
        self.is_occupied = False

    #what methods does a spot need? 
    #it needs -> occupy() and release()

    def occupy(self):
        if not self.is_occupied:
            self.is_occupied = True
            print(f"Spot {self.spot_id} is now occupied")
        else:
            print(f"Spot {self.spot_id} is already occupied")
    
    def release(self):
        if self.is_occupied:
            self.is_occupied = False
            print(f"Spot {self.spot_id} is now available")
        else:
            print(f"Spot {self.spot_id} is available")

    def is_parkingSpot_occupied(self):
        return self.is_occupied #return true if the spot is NOT occupied
    
    def is_size_compatible(self, vehicle: Vehicle):
        
        #check if the parkingspot is compatible with the given vehicle ttype

        if isinstance(vehicle, self.vehicle_type):
            print(f"Spot {self.spot_id} is compatible with {vehicle.__class__.__name__}")
            return True
        return False


In [234]:
import heapq

class ParkingLot:

    #manages multiple spots and assigns the nearest spot
    #holds the number of parking Spots

    def __init__(self, capacity):

        #attributes
        self.capacity = capacity
        self.spots = [] #store the parking lots
        self.available_spots = [] #store the available parking spots
        self.occupied_spots = set() #store the occupied parking spots

    def add_parking_spots(self, parkingSpot: ParkingSpot):
        if len(self.spots) >= self.capacity:
            print("Cannot add more spots, it is at full capacity")
            return
        self.spots.append(parkingSpot)

        #push the new spot with the tuple (distance, ParkingSpot Object )

        heapq.heappush(self.available_spots, (parkingSpot.distance, parkingSpot))
    
    def release_spot(self, parkingSpot: ParkingSpot):

        #no occupied spots
        if len(self.occupied_spots) == 0:
            print(f"There are no occupied spots")
            return
        
        #remove it from the occupied spots
        self.occupied_spots.remove(parkingSpot.spot_id)

        #push it back to our minheap, aka our available spots heap
        heapq.heappush(self.available_spots, (parkingSpot.distance, parkingSpot))

    
    def find_nearest_available_spots(self):
        if not self.available_spots:
            print(f"There are no available parking spots")
            return None
        return self.available_spots[0][1]

    def park(self, vehicle: Vehicle):
        if not self.available_spots:
            print(f"There are no available parking spots")
            return None
        
        nearest_spot = self.find_nearest_available_spots()

        if nearest_spot and nearest_spot.is_size_compatible(vehicle) and not nearest_spot.is_occupied:
            nearest_spot.occupy()
            self.occupied_spots.add(nearest_spot)
            print(f"Vehicle {vehicle.license_plate} is parked at spot {nearest_spot.spot_id}")
            return nearest_spot
        
        print("No suitable Parking Spot")
        return None


In [239]:
parking_lot = ParkingLot(10)

In [244]:
# Create vehicles
compact_car = CompactCar("ABC123")
motorcycle = Motorcycle("MOTO456")


# Create parking lot
parking_lot = ParkingLot(5)


# Create parking spots
spot1 = ParkingSpot(spot_id="A1", distance=5, vehicle_type=CompactCar)
spot2 = ParkingSpot(spot_id="A2", distance=2, vehicle_type=Motorcycle)

# Add spots to parking lot
parking_lot.add_parking_spots(spot1)
parking_lot.add_parking_spots(spot2)

# Park vehicles
parking_lot.park(compact_car)  # Should park in A1
parking_lot.park(motorcycle)  # Should park in A2


No suitable Parking Spot
Spot A2 is compatible with Motorcycle
Spot A2 is now occupied
Vehicle MOTO456 is parked at spot A2


# OOD Problem 2: Design a Coffee Vending Machine

In [ ]:
class Drink:

    def __init__(self, name, ingredients, price, size):
        
        #name
        self.name = name
        #ingredients
        self.ingredients = ingredients #coffee, milk, water, etc

        #price
        self.price = price

        #size
        self.size = size

    
    def get_name(self):
        return self.name
    
    def get_ingredients(self):
        return self.ingredients
    
    def get_price(self):
        return self.price
    
    def get_size(self):
        return self.size
    

class Espresso(Drink):

    def __init__(self, size="Medium"):

        ingredients = {"coffee": 5, "water": 2}
        price = 2.50

        super().__init__("Espresso", ingredients, price, size)

class Cappuccino(Drink):

    def __init__(self, size='Medium'):

        ingredients = {"coffee": 10, "water": 1}
        price = 3.75

        super().__init__("Cappuchino", ingredients, price, size)

class Customization:

    def __init__(self, extra_shot = False, extra_milk = False, sugar_level = 0):
        
        #we are going to have extra shot, extra milk, sugar_level

        self.extra_shot = extra_shot
        self.extra_milk = extra_milk
        self.sugar_level = sugar_level #(0: none to 3 (extra ))

    
    def apply_customization(self, drink: Drink):

        if self.extra_shot:
            drink.ingredients[self.extra_shot] = True

        if self.extra_milk:
            drink.ingredients[self.extra_milk] = True
        

In [266]:
a = Espresso()
b = Cappuccino()

In [263]:
a. get_ingredients()

{'coffee': 5, 'water': 2}